# Pre-process our tennis data set so we can feed in the data to ML and DL model

* 

In [ ]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline



In [ ]:
sqlContext

In [ ]:
# read in our ATP data set and parse tournament dates
matches_orig = pd.read_csv('atpdata/ATP.csv', parse_dates=["tourney_date"])

# we will get tournament info from another dataset
tourney_orig= pd.read_csv('a-large-tennis-dataset-for-atp-and-itf-betting/all_tournaments.csv')

# Data Cleaning

In [ ]:
# make all column names lower case so it's easier to remember
matches = matches_orig
matches.columns = [col.strip().lower() for col in matches.columns]

# these columns don't have much data from our EDA so we can't impute. Let's drop them
drop_columns = ["draw_size","loser_entry", "winner_entry", "loser_seed", "winner_seed"]
matches = matches.drop(drop_columns, axis=1)

# we only care about professional tournaments since we are prediction grand slams so let's filter out non-professional tournaments
matches = matches[~matches_all.tourney_level.isin(["C", "S", "D"])]

# ATP was formed in 1972. Federer turned Pro in 1998, we will only look at matches since January 1998
matches = matches[matches.tourney_date > datetime.datetime(1997, 12, 31)].copy()

# tourney_id is actually a little bit mis-leading - it has the format of {year}-{tourney_id}
# let's correct this
# matches = matches.rename({"tourney_id": "year_tourney_id"}, axis=1)
# matches["tourney_year"] = matches.year_tourney_id.apply(lambda x: x.split("-")[0])
# matches["tourney_id"] = matches.year_tourney_id.apply(lambda x: x.split("-")[1])
# matches = matches.drop("year_tourney_id", axis=1)


In [5]:
# keys for the data are player names and tournament names - these are strings
# we also have come categorical columns - ie loser_hand, winner_hand, surface, tourney_level, 
# let's convert any non-numerical column data into lower case and strip

# first let's print one of the columns
print("Before lowering...")
print(matches[:5].loser_hand)

lower_columns = [col for col, dt in matches.dtypes.items() if dt == np.object]
for col in lower_columns:
    matches[col] = matches[col].str.strip()
    matches[col] = matches[col].str.lower()

# check to make sure we've done this correctly
print("After lowering...")
matches[:5].loser_hand

Before lowering...


NameError: name 'matches' is not defined

In [ ]:
matches.info()

# Impute Missing Data

NOTE: height & weight let's do this later

In [ ]:
## Some matches are misssing minutes

In [3]:
# we are missing some matches's minutes (ie, length of the match)
# we can impute this - since matches length might depend on the tournament (ie, surface) 
# and whether the tournament is best of 3 or 5 - we will use this inforamation 
# to impute by using the mean of match minutes for that tournament
tids = {id for id in matches[matches.minutes.isnull()].tourney_id}
for tid in tids:
    matches.loc[(matches.minutes.isnull()) & (matches.tourney_id == tid), "minutes"] = \
             matches[(matches.minutes.notnull()) & (matches.tourney_id == tid)].minutes.mean()

NameError: name 'matches' is not defined

In [76]:
tids

{'1998-604',
 '1999-604',
 '2000-96',
 '2004-96',
 '2015-311',
 '2015-314',
 '2015-315',
 '2015-316',
 '2015-319',
 '2015-321',
 '2015-328',
 '2015-329',
 '2015-337',
 '2015-341',
 '2015-352',
 '2015-414',
 '2015-418',
 '2015-421',
 '2015-422',
 '2015-429',
 '2015-438',
 '2015-439',
 '2015-440',
 '2015-500',
 '2015-5014',
 '2015-540',
 '2015-560',
 '2015-568',
 '2015-573',
 '2015-6003',
 '2015-605',
 '2015-6116',
 '2015-6242',
 '2015-6718',
 '2015-6967',
 '2015-741',
 '2015-747'}

In [40]:
type(matches[matches.minutes.isnull()])

pandas.core.frame.DataFrame

In [45]:
for index, item in matches[matches.minutes.isnull()].iteritems():
    print(f'index {index} item {item} {item.loc["tourney_id"]}')

KeyError: 'tourney_id'